In [0]:
%run ../python/data_provider

In [0]:
import time
def test_data_provider(share, share_file, sharing_identifier, catalog):
  
  def create_recipient(share_file, sharing_identifier, catalog):
    if share_file != "":
      return DeltaShareRecipient(share_file_loc=share_file, catalog=catalog)
    elif sharing_identifier != "":
      return DeltaShareRecipient(share_file, provider_sharing_identifier=sharing_identifier, catalog=catalog)
    else:
      raise Exception()
  
  def test_discover():
    dsr = create_recipient(share_file, sharing_identifier, catalog)
    dsr.discover()

  def test_sync_remotely():
    dsr = create_recipient(share_file, sharing_identifier, catalog)
    dsr.share_sync(share, cache_locally=False, refresh_incrementally=False, clear_previous_cache=True, clear_sync_history=False,\
                  primary_keys = {'db1.table1':'id', 'db1.table2':'idx'}, num_threads=3)

  def test_sync_remotely_hms():
    if sharing_identifier == "":
      dsr = create_recipient(share_file, sharing_identifier, "hive_metastore")
    else:
      dsr = create_recipient(share_file, sharing_identifier, catalog)
    dsr.share_sync(share, cache_locally=False, refresh_incrementally=False, clear_previous_cache=True, clear_sync_history=False,\
                  primary_keys = {'db1.table1':'id', 'db1.table2':'idx'}, num_threads=3)

  def test_sync_full():
    dsr = create_recipient(share_file, sharing_identifier, catalog)
    dsr.share_sync(share, cache_locally=True, refresh_incrementally=False, clear_previous_cache=True, clear_sync_history=False,\
                  primary_keys = {'db1.table1':'id', 'db1.table2':'idx'}, num_threads=3)

  def test_sync_full_clear():
    dsr = create_recipient(share_file, sharing_identifier, catalog)
    dsr.share_sync(share, cache_locally=True, refresh_incrementally=False, clear_previous_cache=True, clear_sync_history=False,\
                  primary_keys = {'db1.table1':'id', 'db1.table2':'idx'}, num_threads=3)

  def test_sync_incremental():
    dsr = create_recipient(share_file, sharing_identifier, catalog)
    dsr.share_sync(share, cache_locally=True, refresh_incrementally=True, clear_previous_cache=False, clear_sync_history=False,\
                  primary_keys = {'db1.table1':'id', 'db1.table2':'idx'}, num_threads=3)
  
  spark.sql(f"drop catalog if exists amrali_d2d cascade;")
  if sharing_identifier =="":
    spark.sql(f"create catalog if not exists {catalog}")
  test_discover()
  test_sync_remotely()
  test_sync_remotely_hms()
  test_sync_full()
  test_sync_incremental()
  print("------------------------------------------------------")
  print("--")
  print("-- Run some data changes on the source before running incremental updates again")
  print("--")
  print("------------------------------------------------------")
  time.sleep(60) #allow sometime to run some data changes
  test_sync_incremental()

#test_data_provider(share="amr_share", share_file = '/dbfs/FileStore/tables/amr_azure_share.share', sharing_identifier="", catalog="amrali_d2o")
test_data_provider(share="amr_share", share_file="", sharing_identifier='azure:eastus2:b86c6879-8c55-4e70-a585-18d16a4fa6e9', catalog="amrali_d2d")

#test_data_provider(share_file = '/dbfs/FileStore/tables/open_datasets.share', catalog="amrali_d2o")